In [1]:
import os
import numpy as np
from dotenv import load_dotenv
from openai import AzureOpenAI
from pprint import pprint
from utils.prompts.prompt import *
from utils.prompts.Schema_Prompt_v1 import *
from utils.postgres_functions import *

In [2]:
load_dotenv()

True

In [3]:
def calculate_confidence_score(logprobs: list) -> float:
    probs = []
    # iterate through the token predictions and get the linear probs [0;1]
    for logprob in logprobs:
        probs.append(np.round(np.exp(logprob)*100, 2))
    # calculate the confidence
    return round(np.mean(probs), 2)

In [4]:
client = AzureOpenAI(
    api_key = str(os.environ['azure_api_key']),
    api_version = "2024-03-01-preview",
    azure_endpoint = str(os.environ['azure_api_endpoint'])
)

In [5]:
# QUERY = "Forgalomban van a novocetrin?"
# QUERY = "Milyen ciszplatin hatóanyagú gyógyszert tudok filmtablettás gyógyszerként kiírni?"
# QUERY = "Milyen ciszplatin hatóanyagú gyógyszert tudok kiírni?"
# QUERY = "Mennyibe kerül a syntroxine?"
# QUERY = "Milyen gyógyszert lenne érdemes felírni láz esetén?"
# QUERY = "Milyen kiszerelésben kapható az egilok tabletta?"
# QUERY = "Ki forgalmazza a perdox oldatot?"
QUERY = "Mivel lehet kiváltani a Matrifent?"
# QUERY = "Ki írhatja fel a letrozol pharcet?"
# QUERY = "Milyen betegségek esetén alkalmazható a Brusimp?"
# QUERY = "Mi a calcium fluoratum ATC kódjának értelmezése?"

In [26]:
QUERY = "Mire használják a harmonetet?"

In [27]:
sql_gen_messages = [
                {"role": "system", "content" : sql_system_message.format(system_prompt=SQL_SYSTEM_PROMPT, 
                                                                    table_info=SCHEMA_PROMPT)},
                {"role": "user", "content": question_message.format(question=QUERY)}
                ]

In [32]:
full_response = client.chat.completions.create(
    model="TextToSql_Pupha",
    messages=sql_gen_messages,
    temperature=0.0,
    max_tokens=1024,
    logprobs=True
)
pprint(full_response.choices[0].message.content)

('SELECT BNOKODOK.LEIRAS\n'
 'FROM BNOKODOK\n'
 'JOIN BNOHOZZAR ON BNOKODOK.ID = BNOHOZZAR.BNO_ID\n'
 'JOIN EUPONTOK ON BNOHOZZAR.EUPONT_ID = EUPONTOK.ID\n'
 'JOIN EUINDIKACIOK ON EUPONTOK.ID = EUINDIKACIOK.EUPONT_ID\n'
 "WHERE EUINDIKACIOK.LEIRAS ILIKE '%harmonet%'\n"
 'LIMIT 10')


In [8]:
pprint(full_response)

ChatCompletion(id='chatcmpl-9GtuXIJCmJ8ENjF3I8THRVGMxjVE7', choices=[Choice(finish_reason='stop', index=0, logprobs=ChoiceLogprobs(content=[ChatCompletionTokenLogprob(token='SELECT', bytes=[83, 69, 76, 69, 67, 84], logprob=-0.011682977, top_logprobs=[]), ChatCompletionTokenLogprob(token=' G', bytes=[32, 71], logprob=-0.2956411, top_logprobs=[]), ChatCompletionTokenLogprob(token='Y', bytes=[89], logprob=-2.5345586e-05, top_logprobs=[]), ChatCompletionTokenLogprob(token='OG', bytes=[79, 71], logprob=-0.0003894431, top_logprobs=[]), ChatCompletionTokenLogprob(token='YS', bytes=[89, 83], logprob=-6.704273e-07, top_logprobs=[]), ChatCompletionTokenLogprob(token='Z', bytes=[90], logprob=0.0, top_logprobs=[]), ChatCompletionTokenLogprob(token='.O', bytes=[46, 79], logprob=-0.6194805, top_logprobs=[]), ChatCompletionTokenLogprob(token='EP', bytes=[69, 80], logprob=-4.3202e-07, top_logprobs=[]), ChatCompletionTokenLogprob(token='_NE', bytes=[95, 78, 69], logprob=-3.917321e-05, top_logprobs=[]),

In [32]:
logprobs = [x.logprob for x in full_response.choices[0].logprobs.content]
calculate_confidence_score(logprobs)

94.39

In [33]:
db_connection = connect_to_db(str(os.environ['db_host']), str('pupha'), str(os.environ['db_user']), str(os.environ['db_password']))
query_answer = read_db(full_response.choices[0].message.content,db_connection)
close_connection(db_connection)

Connection established


In [34]:
query_answer

""


In [58]:
db_connection = connect_to_db(str(os.environ['db_host']), str("pupha"), str(os.environ['db_user']), str(os.environ['db_password']))
print(read_db('SELECT OEP_NEV FROM GYOGYSZ WHERE EGYENID IN (SELECT EGYENID FROM GYOGYSZ WHERE OEP_NEV LIKE "%MATRIFENT%")',db_connection))
close_connection(db_connection)

Connection established


UndefinedColumn: column "%MATRIFENT%" does not exist
LINE 1: ...N (SELECT EGYENID FROM GYOGYSZ WHERE OEP_NEV LIKE "%MATRIFEN...
                                                             ^


In [36]:
query_answer.drop_duplicates()

,0
0,Teva Gyógyszergyár Zártkörűen Működő Részvényt...


In [14]:
messages = [
                {"role": "system", "content" : text_system_message.format(system_prompt=DB_TO_TEXT_SYSTEM_PROMPT, 
                                                                    sql_data=query_answer.drop_duplicates())},
                {"role": "user", "content": question_message.format(question=QUERY)}
            ]

In [15]:
response = client.chat.completions.create(
    model="TextToSql_Pupha",
    messages=messages,
    temperature=0.0,
    max_tokens=1024,
    logprobs=True
)
pprint(response.choices[0].message.content)

'Az Enbrel hatóanyaga az etanercept.'
